In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
os.chdir('/Users/ianjeong/Desktop/ML/great_cappsby/crypto-currency/data')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/ianjeong/Desktop/ML/great_cappsby/crypto-currency/data'

In [2]:
bitcoin = pd.read_csv('../data/bitcoin_daily.csv')

In [6]:
bitcoin['Daily_Change_Ind'] = bitcoin['Daily_Change'].apply(lambda x: indicator(x))

In [3]:
def indicator(x):
    if x == None:
        return None
    elif x > 0:
        return 1
    else:
        return 0

In [4]:
def daily_change(df):
    df['Daily_Change'] = None
    for row in df.itertuples():
        if row.Index == 0:
            continue
        df.loc[row.Index, 'Daily_Change'] = row.Close - df.loc[row.Index - 1, 'Close']

In [5]:
daily_change(bitcoin)

In [7]:
bitcoin.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Daily_Change,Daily_Change_Ind
0,2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800,None,NaN
1,2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200,-32.894013,0.0
2,2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700,-29.644012,0.0
3,2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600,14.108002,1.0
4,2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100,-10.082978,0.0


In [8]:
def expo_moving_average(df, period):
    '''
    Calculate the exponential moving average.
    
    Input:
        df: a pandas dataframe
        period (int): time period
    Returns:
        rv: a pd series of exponential moving averages
    '''
    col_name = f'EMA_{period}' 
    rv = pd.Series([None] * df.shape[0])
    weight_0 = 2 / (period + 1) 
    weight_1 = 1 - weight_0
    for row in df.itertuples():
        if row.Index < period - 1:
            continue
        elif row.Index == period - 1:
            simple_average = df.loc[:period - 1, 'Close'].sum() / period
            #df.loc[row.Index, col_name] = simple_average
            rv[row.Index] = simple_average
        else:
            #ema = row.Close * weight_0 + df.loc[row.Index - 1, col_name] * weight_1
            ema = row.Close * weight_0 + rv[row.Index - 1] * weight_1
            #df.loc[row.Index, col_name] = ema
            rv[row.Index] = ema
    return rv

In [9]:
def moving_average_convergence_divergence(df):
    '''
    Find the moving average convergence divergence, which is equal to the
    difference between the 12-period exponential moving average and
    26-period exponential moving average.
    
    Modifies the dataframe in place.
    
    Input:
        df: a pd dataframe
    '''
    col_name = 'MACD'
    ema_12 = expo_moving_average(bitcoin, 12)
    ema_26 = expo_moving_average(bitcoin, 26)
    df['MACD'] = ema_12 - ema_26

In [10]:
moving_average_convergence_divergence(bitcoin)

In [11]:
def proc(df, period):
    '''
    Calculate the price rate of change.
    
    Input:
        df: a pandas dataframe
        period (int): time period
    '''
    
    col_name = f'PROC_{period}'
    df[col_name] = None
    for row in df.itertuples():
        if row.Index < period:
            continue
        prev_idx = row.Index - period
        df.loc[row.Index, col_name] = (df.loc[row.Index, 'Close'] - df.loc[prev_idx, 'Close']) / df.loc[prev_idx, 'Close']

In [12]:
proc(bitcoin, 3)
proc(bitcoin, 5)
proc(bitcoin, 10)

In [13]:
def williams_percentage_range(df):
    '''
    Find the Williams Percentage Range. Modifies the df in place. 
    
    Input:
        df: a pd dataframe
    '''
    col_name = 'wpr'
    df[col_name] = None
    for row in df.itertuples():
        if row.Index < 14:
            continue
        max_14 = df.loc[row.Index - 14:row.Index - 1, 'Close'].max()
        min_14 = df.loc[row.Index - 14:row.Index - 1, 'Close'].min()
        wpr = (max_14 - row.Close) / (max_14 - min_14) * -100
        df.loc[row.Index, col_name] = wpr

In [14]:
williams_percentage_range(bitcoin)

In [15]:
def stochastic_oscillator(df):
    '''
    Find the stochastic oscillator. Modifies the df in place.
    '''
    col_name = 'sto_os'
    df[col_name] = None
    for row in df.itertuples():
        if row.Index < 14:
            continue
        max_14 = df.loc[row.Index - 14:row.Index - 1, 'Close'].max()
        min_14 = df.loc[row.Index - 14:row.Index - 1, 'Close'].min()
        so = (row.Close - min_14) / (max_14 - min_14) * 100
        df.loc[row.Index, col_name] = so

In [16]:
stochastic_oscillator(bitcoin)

In [17]:
bitcoin.head(30)

,Date,Open,High,Low,Close,Adj Close,Volume,Daily_Change,Daily_Change_Ind,MACD,PROC_3,PROC_5,PROC_10,wpr,sto_os
0,2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800,None,NaN,NaN,None,None,None,None,None
1,2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200,-32.894013,0.0,NaN,None,None,None,None,None
2,2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700,-29.644012,0.0,NaN,None,None,None,None,None
3,2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600,14.108002,1.0,NaN,-0.105896,None,None,None,None
4,2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100,-10.082978,0.0,NaN,-0.06036,None,None,None,None
5,2014-09-22,399.100006,406.915985,397.130005,402.152008,402.152008,24127600,3.330994,1.0,NaN,0.018632,-0.12066,None,None,None
6,2014-09-23,402.092010,441.557007,396.196991,435.790985,435.790985,45099500,33.638977,1.0,NaN,0.065754,0.026743,None,None,None
7,2014-09-24,435.751007,436.112000,421.131989,423.204987,423.204987,30627700,-12.585998,0.0,NaN,0.06114,0.071959,None,None,None
8,2014-09-25,423.156006,423.519989,409.467987,411.574005,411.574005,26814400,-11.630982,0.0,NaN,0.023429,0.00653,None,None,None
9,2014-09-26,411.428986,414.937988,400.009003,404.424988,404.424988,21460800,-7.149017,0.0,NaN,-0.071975,0.014051,None,None,None


In [18]:
bitcoin.to_csv('bitcoin_daily_modified.csv')

In [19]:
gold = pd.read_csv('../data/gold_modified.csv')
sp500 = pd.read_csv('../data/sp_500.csv')

In [20]:
sp500.head()

,Date,Close/Last,Open,High,Low,sp500_label
0,5/13/2022,4023.89,3963.90,4038.88,3963.90,1
1,5/12/2022,3930.08,3903.95,3964.80,3858.87,1
2,5/11/2022,3935.18,3990.08,4049.09,3928.82,0
3,5/10/2022,4001.05,4035.18,4068.82,3958.17,0
4,5/9/2022,3991.24,4081.27,4081.27,3975.48,1


In [21]:
tweets = pd.read_csv('../data/bitcoin_daily_tweets.csv')
google = pd.read_csv('../data/bitcoin_google_trends.csv')

In [22]:
gold = gold.rename(columns = {'label':'gold_label', 'Close/Last':'gold_close'})
sp500 = sp500.rename(columns = {'Close/Last':'sp500_close'})
gold = gold.loc[:, ['Date', 'gold_close', 'gold_label']]
sp500 = sp500.loc[:, ['Date', 'sp500_close', 'sp500_label']]

In [23]:
tweets = tweets.rename(columns={'date':'Date'})
google = google.rename(columns={'date':'Date', 'bitcoin':'goog_trend_score'})

In [24]:
# modifying date format
gold['Date'] = pd.to_datetime(gold['Date'])
gold['Date'] = gold['Date'].dt.strftime('%Y-%m-%d')
sp500['Date'] = pd.to_datetime(sp500['Date'])
sp500['Date'] = sp500['Date'].dt.strftime('%Y-%m-%d')

In [25]:
gold_merge = bitcoin.dropna().merge(gold, on = 'Date')
sp500_merge = gold_merge.merge(sp500, on = 'Date')
g_merge = sp500_merge.merge(google, on = 'Date')
altogether = g_merge.merge(tweets, on = 'Date')

In [26]:
altogether

,Date,Open,High,Low,Close,Adj Close,Volume,Daily_Change,Daily_Change_Ind,MACD,...,likes_count_weekly_diff,compound_weighted_avg7,compound_weighted_daily_diff,compound_weighted_weekly_diff,pos_weighted_avg7,pos_weighted_daily_diff,pos_weighted_weekly_diff,neg_weighted_avg7,neg_weighted_daily_diff,neg_weighted_weekly_diff
0,2014-10-13,377.920990,397.226013,368.897003,390.414001,390.414001,35221400,11.86499,1.0,-15.70184,...,-10.857143,0.155527,0.475045,-0.062282,0.113330,-0.031695,-0.076025,0.025495,-0.130000,-0.025495
1,2014-10-14,391.691986,411.697998,391.324005,400.869995,400.869995,38491500,10.455994,1.0,-11.895282,...,20.571429,0.117707,-0.084223,-0.108685,0.111784,0.095745,0.021266,0.037595,0.084700,0.047105
2,2014-10-15,400.954987,402.226990,388.765991,394.773010,394.773010,25267100,-6.096985,0.0,-9.263747,...,29.714286,0.120353,0.204298,0.092966,0.118112,-0.032954,-0.018016,0.042944,-0.047263,-0.005506
3,2014-10-17,382.756012,385.477997,375.389008,383.757996,383.757996,13600700,1.201996,1.0,-6.948665,...,-33.285714,0.195819,0.314941,0.332442,0.146115,0.095926,0.049906,0.042944,-0.037437,-0.042944
4,2014-10-20,389.230988,390.084015,378.252014,382.845001,382.845001,16419000,-6.700989,0.0,-3.828066,...,40.428571,0.211852,-0.194504,-0.169295,0.110018,-0.102292,-0.090338,0.023115,-0.039667,-0.023115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1573,2021-02-04,37475.105469,38592.175781,36317.500000,36926.066406,36926.066406,68838074392,-546.023438,0.0,872.62153,...,-21291.571429,0.352018,-0.110210,-0.032645,0.120115,-0.067500,-0.034914,0.022393,-0.005790,-0.005062
1574,2021-02-05,36931.546875,38225.906250,36658.761719,38144.308594,38144.308594,58598066402,1218.242188,1.0,1079.249152,...,121844.714286,0.361516,-0.091800,-0.133943,0.125647,0.035003,-0.005443,0.025481,0.043169,0.035020
1575,2021-02-08,38886.828125,46203.929688,38076.324219,46196.464844,46196.464844,101467222687,7293.023438,1.0,2139.116881,...,264041.142857,0.266808,0.178233,-0.026590,0.113628,0.008827,-0.020424,0.040177,-0.025353,-0.008243
1576,2021-02-09,46184.992188,48003.722656,45166.960938,46481.105469,46481.105469,91809846886,284.640625,1.0,2668.203824,...,1787.285714,0.236172,0.132733,0.136780,0.105731,0.036097,0.023570,0.042145,-0.008810,-0.019021


In [27]:
altogether.to_csv('../data/merged_datasets.csv')